In [ ]:
!pip install -q transformers arabert preprocess
!pip install -q stanza
!pip install -q gensim
!pip install -q flair
!pip install -q lang-trans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.1 MB/s eta 0:00:0000:0100:

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess import ArabertPreprocessor
import stanza

import numpy as np
import pickle
import os
from tqdm import tqdm
from gensim.models import FastText
from flair.data import Sentence
from flair.embeddings import CharacterEmbeddings, StackedEmbeddings
from lang_trans.arabic import buckwalter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import json

In [ ]:
# loaded_weights = np.load('/content/drive/MyDrive/NLP Pro/features/arabic_char_embeddings_256.npy')

In [ ]:
# len(loaded_weights)

In [ ]:
padded_path = "/kaggle/input/arabicia/padded.pkl"
val_path = "/kaggle/input/arabicia/padded_val.pkl"

diacritic2id_path = "/kaggle/input/arabicia/diacritic2id.json"
idx2char_path = "/kaggle/input/arabicia/idx2char.json"
char2idx_path = "/kaggle/input/arabicia/char2idx.json"

char_embeddings_path = "/kaggle/input/char-embeddings/keras/default/1/embedding_matrix.npy"

In [ ]:
def load_data_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        x_ids = [c for c in text_seq]
        y_ids = [t for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return np.array(X), np.array(y)

In [ ]:
with open(diacritic2id_path, 'r', encoding='utf-8') as f:
    diacritic2id = json.load(f)

id2label = {v: k for k, v in diacritic2id.items()}

with open(char2idx_path, 'r', encoding='utf-8') as f:
    char2idx = json.load(f)

with open(idx2char_path, 'r', encoding='utf-8') as f:
    idx2char = json.load(f)

idx2char['1'] = '?'
value = char2idx.pop('\uFFFD')
char2idx['?'] = value

In [ ]:
sentences, tashkeel_sequences = load_data_pickle(padded_path)

In [ ]:

print("Sentences:", len(sentences))
print("Example sentence:", sentences[0][0])
print("Example tashkeel sequence:", tashkeel_sequences[0][0])
print("Example char2idx size:", len(char2idx))
print("Example diacritic2id size:", len(diacritic2id), diacritic2id[tashkeel_sequences[0][0]])

Sentences: 80254
Example sentence: و
Example tashkeel sequence: َ
Example char2idx size: 54
Example diacritic2id size: 15 0


# **Features**

In [ ]:
arabert_model_name = "aubmindlab/bert-base-arabertv02"
bert_tokenizer = AutoTokenizer.from_pretrained(arabert_model_name)
bert_model = AutoModel.from_pretrained(arabert_model_name)
bert_model.eval()
arabert_prep = ArabertPreprocessor(model_name=arabert_model_name)

In [ ]:
# # Model name (AraELECTRA large or small)
# MODEL_NAME = "aubmindlab/araelectra-base-discriminator"

# # Load tokenizer and model
# electra_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# electra_model = AutoModel.from_pretrained(MODEL_NAME)

# # Set to evaluation mode
# electra_model.eval()


In [ ]:
# stanza.download('ar')
# nlp = stanza.Pipeline("ar", processors="tokenize,pos", use_gpu=False)

In [ ]:
# vocab_size = len(char2idx) + 1
# embedding_dim = 128
# char_embedding = nn.Embedding(num_embeddings=vocab_size,
#                               embedding_dim=embedding_dim,
#                               padding_idx=0)

In [ ]:
def remove_pads(sentence, tashkeel):
    chars = []
    tashkeelat = []
    for i in range(len(sentence)):
        if sentence[i] != '<PAD>':
            chars.append(sentence[i])
            tashkeelat.append(tashkeel[i])
    sentence_text = "".join(chars)
    text = sentence_text.replace('\uFFFD', '?')
    return text, tashkeelat

In [ ]:
new_sentences = []
new_tashkeel = []
for i in range(len(sentences)):
    text, tashkeel = remove_pads(sentences[i], tashkeel_sequences[i])
    new_sentences.append(text)
    new_tashkeel.append(tashkeel)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print(f"🚀 GPU Detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory Usage: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
bert_model.to(device)

🚀 GPU Detected: Tesla T4
   Memory Usage: 0.51 GB


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(64000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

# **AraBERT**

In [ ]:
def get_arabert_embeddings(sentence: str):

    tokens = bert_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    tokens = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        output = bert_model(**tokens)

    emb = output.last_hidden_state.squeeze(0).cpu()
    token_list = bert_tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])

    return emb.numpy(), token_list

In [ ]:
emb, toks = get_arabert_embeddings(new_sentences[0])
print("Embedding shape:", type(emb))
print("Tokens:", toks)

Embedding shape: <class 'numpy.ndarray'>
Tokens: ['[CLS]', 'ولو', 'جمع', 'ثم', 'علم', 'ترك', 'ركن', 'من', 'الأولى', 'بطل', '##تا', 'ويعيد', '##هما', 'جامعا', '،', 'أو', 'من', 'الثانية', '،', 'فإن', 'لم', 'يطل', 'تدارك', '،', 'وإلا', 'فب', '##اط', '##لة', 'ولا', 'جمع', '،', 'ولو', 'جهل', 'أعاد', '##هما', 'لوقت', '##يهم', '##ا', '[SEP]']


In [ ]:
# tokens = []
# bert_embeddings = []

# for sent in sentences:
#   emb, tok = get_arabert_embeddings(sentences,device)
#   bert_embeddings.append(emb)
#   tokens.append(tok)

# **POS Tagging**

In [ ]:
# def extract_pos_tags(sentence: str):
#     doc = nlp(sentence)
#     pos_tags = []
#     for sent_obj in doc.sentences:
#         for word in sent_obj.words:
#             pos_tags.append(word.upos)
#     return pos_tags

In [ ]:
# print(extract_pos_tags(sentences[0])[:10])

# **Char Level Embedding**

In [ ]:
def char_encode(sentence: str):
    return [char2idx[c] for c in sentence]

def char_embed(sentence: str):
    ids = torch.tensor(char_encode(sentence))
    return char_embedding(ids).detach().numpy()

In [ ]:
# print("Char embedding:", char_embed('ر'))

# **Fast Text Word Embedding**

In [ ]:
# def get_arabic_tokens(corpus):
#     data = [sentence.split() for sentence in corpus]
#     return data


In [ ]:
# def train_fasttext_arabic(corpus, embedding_size=100, window_size=5, min_count=3, epochs=50, model_path="./models/ft_arabic_model"):
#     """
#     Train a FastText model on raw Arabic tokens
#     """
#     data = get_arabic_tokens(corpus)

#     # Initialize FastText model
#     ft_model = FastText(
#         vector_size=embedding_size,
#         window=window_size,
#         min_count=min_count,
#         workers=4,
#         sg=1  # Skip-gram
#     )
#     ft_model.build_vocab(corpus_iterable=data)
#     ft_model.train(corpus_iterable=data, total_examples=len(data), epochs=epochs)

#     # Save model
#     os.makedirs(os.path.dirname(model_path), exist_ok=True)
#     ft_model.save(model_path)
#     print(f"✅ FastText Arabic model saved at {model_path}")

#     return ft_model

In [ ]:
# def extract_fasttext_embeddings_arabic(corpus, ft_model):
#     """
#     Extract FastText embeddings for each word in the corpus
#     Returns a list of sentences, where each sentence is a list of word vectors
#     """
#     data = get_arabic_tokens(corpus)
#     all_embeddings = []

#     for sentence in data:
#         sentence_embeddings = []
#         for word in sentence:
#             vec = ft_model.wv[word]  # FastText handles OOV words via subword info
#             sentence_embeddings.append(vec)
#         all_embeddings.append(sentence_embeddings)

#     return all_embeddings

In [ ]:
# ft_model = train_fasttext_arabic(sentences)

In [ ]:
# extract_fasttext_embeddings_arabic(sentences[0],ft_model)

In [ ]:
# !cp -r "./models/" '/content/drive/MyDrive/NLP/'

# Custom Embeddings

In [ ]:
custom_char_embedding = np.load(char_embeddings_path)

def extract_custom_char_embeddings(char):
    return custom_char_embedding[char2idx[char]]

# **FLAIR Char Embedding**

In [ ]:
# char_embedding = CharacterEmbeddings()

# def extract_char_embeddings(sentence_text, embedding_model=None):

#     if embedding_model is None:
#         embedding_model = char_embedding
#     sentence = Sentence(sentence_text)
#     embedding_model.embed(sentence)

#     return [token.embedding.detach().cpu() for token in sentence]

In [ ]:
# extract_char_embeddings(sentence_text)

# **AraELECTRA**

In [ ]:
# def get_araelectra_embeddings(sentence, model, tokenizer, device="cpu"):
#     """
#     Get token-level embeddings from AraELECTRA
#     Returns a list of sentence embeddings (list of token embeddings)
#     """
#     model.to(device)
#     inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     # Get outputs (last hidden state)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         last_hidden_state = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
#     # Remove batch dimension and convert to list of embeddings per token
#     token_embeddings = last_hidden_state.squeeze(0)  # [seq_len, hidden_size]
#     return token_embeddings

In [ ]:
# # Example: using your Buckwalter-transliterated sentences or cleaned Arabic
# ara_embeddings = get_araelectra_embeddings(sentences[0], electra_model, electra_tokenizer, device)

# print("Number of tokens in first sentence:", ara_embeddings.shape[0])
# print("Embedding dimension:", ara_embeddings.shape[1])
# print(ara_embeddings)

# **Buckwalter Translation**

In [ ]:
# def get_buckwalter_translation(sentence):
#   return buckwalter.transliterate(sentence)

In [ ]:
# print("Original sentence:", sentences[0])
# print("Buckwalter transliterated:", get_buckwalter_translation(sentences[0]))

In [ ]:
# buckwalter_sentences = []
# for s in sentences:
#   buckwalter_sentences.append(get_buckwalter_translation(s))

# **TF-IDF**

In [ ]:
# def tf_idf_features(sentences, save_path="models/tf_idf_buckwalter.csv"):
#     os.makedirs(os.path.dirname(save_path), exist_ok=True)

#     vectorizer = TfidfVectorizer(lowercase=False)  # Don't lowercase Buckwalter
#     tfidf_matrix = vectorizer.fit_transform(sentences)

#     feature_names = vectorizer.get_feature_names_out()
#     df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
#     df_tfidf.to_csv(save_path, index=False)

#     print(f"✅ TF-IDF features saved at {save_path}")
#     return df_tfidf, vectorizer

In [ ]:
# tfidf_df, tfidf_vectorizer = tf_idf_features(buckwalter_sentences)
# print("TF-IDF shape:", tfidf_df.shape)
# print("Example TF-IDF features:", tfidf_df.columns[:10])

# **BOW**

In [ ]:
# def bow_features(sentences, save_path="models/bow_buckwalter.csv"):
#     os.makedirs(os.path.dirname(save_path), exist_ok=True)

#     vectorizer = CountVectorizer()
#     bow_matrix = vectorizer.fit_transform(sentences)

#     feature_names = vectorizer.get_feature_names_out()
#     df_bow = pd.DataFrame(bow_matrix.toarray(), columns=feature_names)
#     df_bow.to_csv(save_path, index=False)

#     print(f"✅ Bag-of-Words features saved at {save_path}")
#     return df_bow, vectorizer

In [ ]:
# bow_df, bow_vectorizer = bow_features(buckwalter_sentences)
# print("BoW shape:", bow_df.shape)
# print("Example BoW features:", bow_df.columns[:10])

# **Labeling**

In [ ]:
def get_tashkeel_sequence(index: int):
    return new_tashkeel[index]

In [ ]:
print(get_tashkeel_sequence(0)[:10])

['َ', 'َ', 'ْ', '', 'َ', 'َ', 'َ', '', 'ُ', 'َّ']


In [ ]:
result_str = ''
for i in range(15):
    result_str += new_sentences[0][i] + new_tashkeel[0][i]
result_str

'وَلَوْ جَمَعَ ثُمَّ عَلِمَ '

# **Full Feature Pipeline**

In [ ]:
def tokens_to_word_embeddings(tokens, embeddings):
    word_embeddings = []
    current_word_embs = []

    for token, emb in zip(tokens, embeddings):
        emb_tensor = torch.tensor(emb) if isinstance(emb, np.ndarray) else emb

        if token.startswith("##"):
            current_word_embs.append(emb_tensor)
        else:
            if current_word_embs:
                word_embeddings.append(torch.mean(torch.stack(current_word_embs), dim=0))
            current_word_embs = [emb_tensor]

    if current_word_embs:
        word_embeddings.append(torch.mean(torch.stack(current_word_embs), dim=0))

    return torch.stack(word_embeddings)


In [ ]:
def zizo_features(sentence: str,
                  sent_index: int,
                  arabert_model=None,
                  arabert_tokenizer=None,
                  fasttext_model=None):

    sentence_vec = []

    tashkeel = get_tashkeel_sequence(sent_index)

    arabert_emb, tokens = get_arabert_embeddings(sentence)
    final_arabert_emb = tokens_to_word_embeddings(tokens, arabert_emb)

    words_raw = sentence.split()
    word_idx = 0
    char_in_word_idx = 0

    emb_dim = final_arabert_emb[0].shape[0]

    for i, char in enumerate(sentence):

        char_emb = extract_custom_char_embeddings(char)
        char_emb_array = np.array(char_emb).flatten()

        if char == ' ':
            bert_vec = np.zeros(emb_dim)

        else:
            bert_vec = final_arabert_emb[word_idx]
            if isinstance(bert_vec, torch.Tensor):
                bert_vec = bert_vec.numpy()

            char_in_word_idx += 1

            if char_in_word_idx == len(words_raw[word_idx]):
                word_idx += 1
                char_in_word_idx = 0

        char_vector = np.concatenate([bert_vec, char_emb_array])
        sentence_vec.append(char_vector)

    return sentence_vec, tashkeel

In [ ]:
f,t = zizo_features(new_sentences[0], 0,bert_model,bert_tokenizer)

In [ ]:
len(f)

137

In [ ]:
len(f[0])

1024

In [ ]:
len(new_sentences[0])

137

In [ ]:
len(t)

137

# Create Embeddings

In [ ]:
import h5py
import numpy as np
import os
from tqdm import tqdm
import shutil
import json
import math
from kaggle.api.kaggle_api_extended import KaggleApi


In [ ]:
import os, json

my_username = "ziad23samer"
my_key = "0dda25fbbd0112eac2ff1848742972a2"

creds = {"username": my_username, "key": my_key}
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(creds, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

print("Login info set!")

Login info set!


In [ ]:

api = KaggleApi()
api.authenticate()

WORKING_FILE = "/kaggle/working/arabic_diacrirization_features.h5"
TEMP_DIR = "/kaggle/tmp/rescue"
DATASET_SLUG = f"{api.get_config_value('username')}/arabic-diacrirization-features"
DATASET_TITLE = "Rescue Data Output"

def upload_checkpoint(current_index, total_len):
    """
    Copies current file to tmp and pushes to Kaggle Dataset.
    """
    filename = os.path.basename(WORKING_FILE)
    dest_path = os.path.join(TEMP_DIR, filename)

    print(f"\n[Checkpoint] Processing upload at index {current_index}...")

    if os.path.exists(TEMP_DIR):
        shutil.rmtree(TEMP_DIR)
    os.makedirs(TEMP_DIR, exist_ok=True)

    shutil.copy(WORKING_FILE, dest_path)

    meta = {
        "title": DATASET_TITLE,
        "id": f"{DATASET_SLUG}-{current_index}",
        "licenses": [{"name": "CC0-1.0"}]
    }
    with open(os.path.join(TEMP_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(meta, f)


    try:
        print(f"Creating NEW dataset: {DATASET_SLUG}-{current_index}")
        api.dataset_create_new(
            folder=TEMP_DIR,
            dir_mode='zip',
            quiet=True
        )
        print("Upload command sent successfully.\n")
    except Exception as e:
        print(f"Upload FAILED: {e}")
        pass

    shutil.rmtree(TEMP_DIR)

In [ ]:
H5_PATH = "/kaggle/working/arabic_diacrirization_features.h5"
CHECKPOINT_INDEX_PATH = "/kaggle/working/arabic_diacrirization_features.txt"

SAVE_EVERY = 100
UPLOAD_EVERY_N_SAVES = 160

def map_tashkeel(char_list):
    return [diacritic2id[c] for c in char_list]

start_index = 0
if os.path.exists(CHECKPOINT_INDEX_PATH):
    with open(CHECKPOINT_INDEX_PATH, "r") as f:
        start_index = int(f.read())
    print(f"Resuming from index: {start_index}")

buffer_features = []
buffer_tashkeel = []
save_counter = 0

if start_index == 0 and os.path.exists(H5_PATH):
    os.remove(H5_PATH)

for i in tqdm(range(start_index, len(new_sentences))):
    sent = new_sentences[i]

    f, t = zizo_features(sent, i, bert_model, bert_tokenizer)

    buffer_features.extend(f)
    t_ids = map_tashkeel(t)
    buffer_tashkeel.extend(t_ids)

    assert len(f) == len(t_ids), "They should be equal"

    if (i + 1) % SAVE_EVERY == 0 or i == len(new_sentences) - 1:

        batch_features = np.array(buffer_features, dtype=np.float16)
        batch_tashkeel = np.array(buffer_tashkeel, dtype=np.int32)

        with h5py.File(H5_PATH, 'a') as hf:
            if 'features' not in hf:
                hf.create_dataset('features', data=batch_features,
                                  maxshape=(None, 1024),
                                  chunks=True,
                                  compression="lzf")
                hf.create_dataset('tashkeel', data=batch_tashkeel,
                                  maxshape=(None,),
                                  chunks=True,
                                  compression="lzf")
            else:
                hf['features'].resize((hf['features'].shape[0] + batch_features.shape[0]), axis=0)
                hf['tashkeel'].resize((hf['tashkeel'].shape[0] + batch_tashkeel.shape[0]), axis=0)

                hf['features'][-batch_features.shape[0]:] = batch_features
                hf['tashkeel'][-batch_tashkeel.shape[0]:] = batch_tashkeel

        with open(CHECKPOINT_INDEX_PATH, "w") as f:
            f.write(str(i + 1))

        buffer_features = []
        buffer_tashkeel = []

        save_counter += 1

        if save_counter >= UPLOAD_EVERY_N_SAVES or (i == len(new_sentences) - 1 and save_counter > 0):
            print(f"\n[Trigger] Reached {save_counter} saves. Uploading and clearing disk...")

            part_filename = f"zizo_part_{i+1}.h5"
            part_path = f"/kaggle/working/{part_filename}"

            shutil.move(H5_PATH, part_path)

            global WORKING_FILE
            WORKING_FILE = part_path

            try:
                upload_checkpoint(i + 1, len(new_sentences))
                if os.path.exists(part_path):
                    os.remove(part_path)
                    print(f"Deleted local file {part_filename} to free space.")

                save_counter = 0
            except Exception as e:
                print(f"Upload failed: {e}")
                shutil.move(part_path, H5_PATH)
                continue

 20%|█▉        | 15998/80254 [03:28<11:34, 92.54it/s]


[Trigger] Reached 160 saves. Uploading and clearing disk...

[Checkpoint] Processing upload at index 16000...
Creating NEW dataset: ziad23samer/arabic-diacrirization-features-16000
Upload command sent successfully.



 20%|█▉        | 16017/80254 [03:37<3:27:31,  5.16it/s]

Deleted local file zizo_part_16000.h5 to free space.


 40%|███▉      | 31993/80254 [07:06<09:10, 87.69it/s]  

...

Deleted local file zizo_part_80000.h5 to free space.


100%|█████████▉| 80253/80254 [18:18<00:00, 80.32it/s]


[Trigger] Reached 3 saves. Uploading and clearing disk...

[Checkpoint] Processing upload at index 80254...
Creating NEW dataset: ziad23samer/arabic-diacrirization-features-80254


100%|██████████| 80254/80254 [18:21<00:00, 72.85it/s]

Upload command sent successfully.

Deleted local file zizo_part_80254.h5 to free space.


In [ ]:
# import pickle
# import os
# from tqdm import tqdm

# DRIVE_CHECKPOINT_PATH = "/kaggle/working/zizo_checkpoint.pkl"

# final_features = []
# final_tashkeel = []

# START_INDEX = 0
# SAVE_EVERY = 100

# # Resume if Drive checkpoint exists
# if os.path.exists(DRIVE_CHECKPOINT_PATH):
#     print("🔁 Resuming from checkpoint...")
#     with open(DRIVE_CHECKPOINT_PATH, "rb") as f:
#         data = pickle.load(f)
#         final_features = data["features"]
#         final_tashkeel = data["tashkeel"]
#         START_INDEX = data["index"] + 1
#     print(f"➡️ Resumed from index: {START_INDEX}")

# for i in tqdm(range(START_INDEX, len(sentences))):
#     sent = sentences[i]

#     f, t = zizo_features(sent, i, bert_model, bert_tokenizer, device)
#     final_features.append(f)
#     final_tashkeel.append(t)

#     if (i + 1) % SAVE_EVERY == 0 or i == len(sentences) - 1:
#         checkpoint_data = {
#             "features": final_features,
#             "tashkeel": final_tashkeel,
#             "index": i
#         }
#         with open(DRIVE_CHECKPOINT_PATH, "wb") as f:
#             pickle.dump(checkpoint_data, f)

#         print(f"💾 Direct checkpoint saved to Drive at index {i}")

In [ ]:
# def extract_all_features(
#     sentence: str,
#     sent_index: int,
#     arabert_model=None,
#     arabert_tokenizer=None,
#     araelectra_model=None,
#     araelectra_tokenizer=None,
#     fasttext_model=None,
#     flair_char_embed=None,
#     buckwalter_enabled=True,
#     device = "cpu"
# ):
#     """
#     Extract all features for a single sentence.

#     Returns a dictionary with:
#     - tokens
#     - ArabERT embeddings (list of token embeddings)
#     - POS tags
#     - Flair char embeddings (list of char-level embeddings)
#     - Tashkeel sequence (diacritics)
#     - Buckwalter transliteration (optional)
#     - FastText embeddings (list of word embeddings)
#     - AraELECTRA sentence embedding
#     """

#     features = {}

#     # --- Tokens + ArabERT embeddings ---
#     arabert_emb, tokens = get_arabert_embeddings(sentence)
#     features["tokens"] = tokens
#     features["arabert_embeddings"] = arabert_emb

#     # --- POS tags ---
#     features["pos"] = extract_pos_tags(sentence)

#     # --- Flair char embeddings ---
#     features["char_embeddings"] = extract_char_embeddings(sentence) if flair_char_embed else None

#     # --- Tashkeel / diacritics ---
#     features["diacritics"] = get_tashkeel_sequence(sent_index)

#     # --- Buckwalter transliteration ---
#     if buckwalter_enabled:
#         from lang_trans.arabic import buckwalter
#         features["buckwalter"] = buckwalter.transliterate(sentence)
#         features['tf-idf']=tfidf_df.loc[sent_index]
#         features['bow']=bow_df.loc[sent_index]

#     # --- FastText embeddings ---
#     if fasttext_model:
#         # split sentence into words
#         words = sentence.split()
#         features["fasttext_embeddings"] = [fasttext_model.wv[word] if word in fasttext_model.wv else None for word in words]

#     # --- AraELECTRA sentence embedding ---
#     if araelectra_model and araelectra_tokenizer:
#         sent_emb = get_araelectra_embeddings(sentence, araelectra_model, araelectra_tokenizer)
#         features["araelectra_embedding"] = sent_emb

#     return features

In [ ]:

features = extract_all_features(sentences[0], 0,bert_model,bert_tokenizer,electra_model,electra_tokenizer,
    ft_model,True,True,device)
print(features)

In [ ]:
print(features.keys())

In [ ]:
def extract_features_for_all(sentences):
    all_features = []
    for i, sent in tqdm(enumerate(sentences), total=len(sentences)):
        feats = extract_all_features(sent, i,bert_model,bert_tokenizer,electra_model,electra_tokenizer,
    ft_model,
    True,
    True,device)
        all_features.append(feats)
    return all_features


# ---- Run feature extraction for the whole dataset ----
full_feature_dataset = extract_features_for_all(sentences)

# ---- Save to file ----
with open("arabic_diacritization_features.pkl", "wb") as f:
    pickle.dump(full_feature_dataset, f)

print("🎯 All features extracted and saved successfully!")
print("📁 Output file: arabic_diacritization_features.pkl")
print("Total samples:", len(full_feature_dataset))

# **AraVec** (still not working)

In [ ]:
# !unzip full_grams_cbow_300_twitter.zip -d aravec_twitter_cbow_300

In [ ]:
# from gensim.models import KeyedVectors

# model_path = "full_grams_cbow_300_twitter.mdl"

# aravec = KeyedVectors.load(model_path)

In [ ]:
# def tokenize(text):
#     text = arabert_prep.preprocess(text)
#     return text.split()

In [ ]:
import shutil, os, json

# --- CONFIGURATION ---
# Keep this exactly as you had it
source_path = "/kaggle/working/zizo_dataset_2.h5"
# ---------------------

# 1. Move file to /kaggle/tmp
# We extract just the name "zizo_dataset_2.h5" to avoid the path error
filename_only = os.path.basename(source_path)
dest_path = os.path.join("/kaggle/tmp/rescue", filename_only)

print(f"Moving {filename_only} to temporary storage...")
os.makedirs("/kaggle/tmp/rescue", exist_ok=True)
shutil.move(source_path, dest_path)

# 2. Create Dataset Metadata
with open('/root/.kaggle/kaggle.json') as f:
    creds = json.load(f)

meta = {
    "title": "Rescue Data Output",
    "id": f"{creds['username']}/rescue-data-output",
    "licenses": [{"name": "CC0-1.0"}]
}
with open('/kaggle/tmp/rescue/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

# 3. Upload to Kaggle
print("Starting upload... This will take a few minutes.")
!kaggle datasets create -p /kaggle/tmp/rescue --dir-mode zip

In [ ]:
len(new_sentences)

80254

In [ ]:
char2idx

{'<PAD>': 0,
 'و': 2,
 'ل': 3,
 ' ': 4,
 'ج': 5,
 'م': 6,
 'ع': 7,
 'ث': 8,
 'ت': 9,
 'ر': 10,
 'ك': 11,
 'ن': 12,
 'ا': 13,
 'أ': 14,
 'ى': 15,
 'ب': 16,
 'ط': 17,
 'ي': 18,
 'د': 19,
 'ه': 20,
 '،': 21,
 'ة': 22,
 'ف': 23,
 'إ': 24,
 'ق': 25,
 'ز': 26,
 'ؤ': 27,
 'ض': 28,
 'ذ': 29,
 'س': 30,
 'خ': 31,
 'ح': 32,
 'ظ': 33,
 'ء': 34,
 '.': 35,
 ':': 36,
 '{': 37,
 '}': 38,
 'ش': 39,
 '[': 40,
 ']': 41,
 'ص': 42,
 '(': 43,
 'غ': 44,
 ')': 45,
 '؛': 46,
 'ئ': 47,
 '؟': 48,
 'آ': 49,
 '-': 50,
 '!': 51,
 '«': 52,
 '»': 53,
 '?': 1}